## Yandex Cloud

Облако Yandex Cloud содержит множество сервисов для выполнения базовых операций. Ниже мы приведём несколько "строительных блоков" в виде кусочков кода на Python, которые вы сможете использовать в своих проектах, например:

* Распознавание текста на картинке
* Рисование картинки по текстовому запросу с Yandex ART
* Работа с языковыми моделями в облаке, чтобы переделать текст:
   - сделать его более вежливым
   - получить краткое содержание текста
   - перевести его на другой язык
   - проверить работу студента
* распознать звук (речь) в текст
* синтезировать речь

Для начала получим файл с секретными ключами для доступа в облако (ключи будут доступны на время интенсива и работы над проектами):

In [ ]:
!wget https://storage.yandexcloud.net/ycpub/keys/.env

Также скачаем ещё полезные файлы, который нам пригодятся дальше по ходу демонстрации:

In [ ]:
!wget https://github.com/yandex-datasphere/jacademy/blob/main/2024/images/slide.jpg?raw=true -O slide.jpg

### Устанавливаем необходимые библиотеки

Для начала надо установить необходимые библиотеки Python, которые понадобятся нам в работе.

In [ ]:
%pip install dotenv yandex_cloud_ml_sdk yandex-speechkit==1.5.0 telebot==0.0.5 gradio openai shwarsutils

> **ВНИМАНИЕ!!!** После установки библиотек рекомендуется зайти в пункт меню **Kernel** -> **Restart Kernel**.

Теперь получаем секретные ключи для работы с облаком из ранее скачанного файла:

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

folder_id  = os.environ['folder_id']
api_key = os.environ['api_key']


### Использование больших языковых моделей

Большие языковые модели позволяют вам совершать различные манипуляции с текстом. Основное, что может модель - это отвечать на ваши вопросы. Соответственно, формулируя вопросы по-разному, вы можете попросить модель придумать что-то, перевести текст, перефразировать его в более короткой или более вежливой форме, и многое другое.

Для работы с моделью создадим объект `gpt`:

In [ ]:
from openai import OpenAI
from IPython.display import Markdown, display

# Выбираем модель, которую хотим использовать
#model = f"gpt://{folder_id}/yandexgpt/rc"
#model = f"gpt://{folder_id}/gemma-3-27b-it/latest"
#model = f"gpt://{folder_id}/gpt-oss-120b/latest"
model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

client = OpenAI(
    base_url="https://rest-assistant.api.cloud.yandex.net/v1",
    api_key=api_key,
    project=folder_id
)

def printx(string):
    display(Markdown(string))

def gpt(x, model=model):
    res = client.responses.create(
        model = model,
        input = x)
    return res.output_text

printx(gpt('Привет! Придумай 10 новых слов для приветствия.'))

Используем модель для преобразования текста в вежливую подробную форму:

In [ ]:
texts = [
    "Мне было лень делать домашку",
    "Я опоздал, потому что проспал"
]

prompt = """Пожалуйста, перепиши текст ниже в тройных обратных кавычках в подробной вежливой 
форме, длиною в несколько предложений, не короче 2000 символов. Вот текст:```{}```"""

for x in texts:
    res = gpt(prompt.format(x))
    printx(f"**{x}**\n{res}\n")

Можем попросить модель сделать что-то более сложное, например, оценить ответ школьника на тесте:

In [ ]:
texts = [
    "Млекопитающие — это типа такие животные, у которых есть шерсть и они рожают детёнышей. Ну и кормят их молоком, потому что у них есть молочные железы. Вроде всё.",
    "Млекопитающие — это класс позвоночных животных, характеризующийся наличием молочных желез, которые позволяют самкам кормить потомство. Большинство млекопитающих также обладают шерстью или волосами, теплокровностью и сложной системой дыхания. Они рождают живых детёнышей (за исключением яйцекладущих, таких как утконос и ехидна) и имеют высокоразвитую нервную систему, что обеспечивает сложное поведение и способность к обучению."
]

prompt = """
Мне нужно проверить ответ школьника на тесте по биологии на вопрос "млекопитающие". Тебе нужно оценить 
ответ ниже в тройных обратных кавычках по трем параметрам: полнота ответа (complete), 
стройность и стиль изложения (style) и соответствие теме (topic). Также выстави общую среднюю оценку за ответ (grade). Все
оценки выставляй по шкале от 1 до 5, возможно, с дробном виде.
Прочитай текст в сообщении ниже и верни ответ в формате JSON.
Верни только JSON, не нужно объяснений или другого текста.
Ответ школьника:
```{}```
"""

for x in texts:
    res = gpt(prompt.format(x)).replace('```','')
    printx(f"**{x}**\n<pre>{res}</pre>")

Использованный тут формат JSON позволяет легко преобразовать ответ из текста в данные Python:

In [ ]:
import json
import pandas as pd

res = []
for x in texts:
    y = gpt(prompt.format(x))
    y = json.loads(y.replace('```',''))
    print(y)
    y['answer'] = x
    res.append(y)
    
df = pd.DataFrame(res)
df


### YandexART

YandexART - это сеть для рисования картинок по текстовому запросу:

In [ ]:
import io
from PIL import Image
from yandex_cloud_ml_sdk import YCloudML

sdk = YCloudML(folder_id=folder_id, auth=api_key)
yart_model = sdk.models.image_generation('yandex-art')

def generate(prompt):
    res = yart_model.run_deferred(prompt)
    res = res.wait()
    return Image.open(io.BytesIO(res.image_bytes))

generate("Сферический конь в вакууме")

Используем возможности программирования, чтобы нарисовать сразу много изображений по алгоритму:

In [ ]:
from shwars.utils import display_images
from tqdm.auto import tqdm

res = []
for c in tqdm(["красным","оранжевым","жёлтым","зелёным","голубым","синим","фиолетовым"]):
    res.append(generate(f"Портрет девушки с {c} цветом волос, с интересом смотрящей в камеру"))

In [ ]:
display_images(res)

### Другие сервисы Yandex Cloud

Для работы с другими сервисами Yandex Cloud надо использовать специальные протоколы общения - REST. Ниже пара функций, которые помогут нам в дальнейшем вызывать разные функции в облаке. Не переживайте, если вы совсем не понимаете, что тут написано.

In [ ]:
import requests
import os

def call_api(url, data):
    headers = { "Authorization" : f"Api-Key {api_key}" }
    return requests.post(url, json=data, headers=headers).json()

def call_api_get(url, data):
    headers = { "Authorization" : f"Api-Key {api_key}" }
    return requests.get(url, headers=headers).json()

### Распознаём текст на картинке

Для распознавания текста на картике служит сервис Yandex Vision OCR. Рассмотрим следующую картинку:

In [ ]:
from PIL import Image

img = Image.open('slide.jpg')
img

Здесь для работы с картинками мы используем специальную библиотеку Python - PIL. Объект `Image` представляет собой картинку, которую можно считывать и записывать из файла, показывать и т.д.

Функция `ocr` ниже берёт такую картинку и возвращает результат распознавания текстовых объектов на ней:

In [ ]:
from io import BytesIO
import base64

def ocr(img):
    buffer = BytesIO()
    img.save(buffer,format="JPEG")
    myimage = buffer.getvalue()

    j = {
      "mimeType": "JPEG",
      "languageCodes": ["*"],
      "model": "page",
      "content": base64.b64encode(myimage).decode('utf-8')
    }
    res = call_api("https://ocr.api.cloud.yandex.net/ocr/v1/recognizeText",j)
    return res

ocr(img)

Чтобы получить только текст, используем такой вариант:

In [ ]:
def ocr_text(img):
    res = ocr(img)
    return res['result']['textAnnotation']['fullText']

txt = ocr_text(img)
print(txt)

Теперь этот текст можно подать на вход gpt-модели, чтобы получить более красивый ответ, отформатированный в формате Markdown:

In [ ]:
prompt = """Ты - научный редактор. Тебе на вход подаётся текст в тройных обратных кавычках, распознанный 
со слайдов презентации. Пожалуйста, приведи этот текст в аккуратный вид и представь его в формате Markdown.
Текст: ```{}```
"""

res = gpt(prompt.format(txt))
print(res)

### Синтез речи

Для синтеза речи будем использовать специальную библиотеку *Yandex SpeechKit*. Мы опишем функцию, которой можно передать на вход текст, а на выходе получить речь:

In [ ]:
from speechkit import model_repository, configure_credentials, creds

# Аутентификация через API-ключ.
configure_credentials(
   yandex_credentials=creds.YandexCredentials(
      api_key=api_key
   )
)

def synthesize(text,voice='jane'):
   model = model_repository.synthesis_model()

   # Задайте настройки синтеза.
   model.voice = voice

   # Синтез речи и создание аудио с результатом.
   result = model.synthesize(text, raw_format=False)
   return result

res = synthesize('Привет, как ты?')
res

Можем скомбинировать синтез с gpt, и получится синтезатор анекдотов:

In [ ]:
res = gpt("Расскажи анекдот про Вовочку")
res = synthesize(res)
res

С фрагментами речи можно выполнять операцию сложения, чтобы смонтировать целую передачу из анекдотов. Также мы используем разные голоса, чтобы было интереснее слушать.

In [ ]:
topics = ['Вовочку','Русского и литовца','Математику']
anecdote_radio = None
for x in topics:
    print(f"Генерирую анекдот про {x}")
    f = synthesize(f'Анекдот про {x}',voice='julia')
    if anecdote_radio:
        anecdote_radio +=f
    else:
        anecdote_radio = f
    p = f"Расскажи анекдот про {x}. На выходе выведи только сам анекдот, без лишнего текста"
    f = synthesize(gpt(p),voice='zahar')
    anecdote_radio += f
    
anecdote_radio

### Распознавание речи

SpeechKit может решать обратную задачу - распознавание речи. Для этого опишем функцию `reco`, и передадим туда сгенерированную нами на прошлом шаге "передачу анекдотов".

In [ ]:
from speechkit.stt import AudioProcessingType

def reco(bin):
    model = model_repository.recognition_model()
    model.model = 'general'
    model.language = 'ru-RU'
    model.audio_processing_type = AudioProcessingType.Full
    result = model.transcribe(bin)
    # можно использовать transcribe_file для распознавания файла с речью
    return ' '.join(x.normalized_text for x in result)

reco(anecdote_radio)

## Выводы

В облаке Yandex Cloud содержится много сервисов, которые можно легко использовать как строительные блоки для реализации типовых задач: синтез/распознавание голоса, OCR, обращение к языковым моделям и рисования картинок. Но вы не можете влиять на то, как работают эти сервисы.